## Set Up Twilio API

In [36]:
! pip install twilio

In [37]:
from google.colab import userdata
from twilio.rest import Client

In [45]:
# Your Twilio account SID and auth token
account_sid = userdata.get("YOUR_TWILIO_ACCOUNT_SID")
auth_token = userdata.get("YOUR_TWILIO_AUTH_TOKEN")

In [46]:
def send_sms(prompt):
    # Simulate the API call to create a Sequential model
    print(f"API Call: Send Message")

    # Invoke API
    client = Client(account_sid, auth_token)
    message = client.messages.create(
        body = prompt,
        from_ = "+18552060350", # From
        to = '+15859538396' # To
    )

    print(message.sid)

    return None

In [47]:
send_sms("set a demo at 9PM today")

API Call: Send Message
SM40d0eaf1f4240f7da677017bd30612dc


## Setup OpenAI

In [68]:
! pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.5/373.5 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.0 MB/s eta 0:00:00


In [69]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [72]:
from openai import OpenAI

class ChatBot:
    def __init__(self, protocol: str):
        self.client = OpenAI(api_key=OPENAI_API_KEY)
        self.protocol = protocol
        self.history = [{"role": "system", "content": self.protocol}]

    def generate_response(self, prompt: str) -> str:
        self.history.append({"role": "user", "content": prompt})

        completion = self.client.chat.completions.create(
            model="gpt-4o",
            messages=self.history
        )

        response = completion.choices[0].message.content
        self.history.append({"role": "assistant", "content": response})

        return response

    def get_history(self) -> list:
        return self.history

In [74]:
# Example usage:
bot = ChatBot(
    protocol="You are a helpful agent."
)
response = bot.generate_response("What is 1+2?")
print(response)

1 + 2 equals 3.


In [78]:
import re
from typing import Any, Dict, Optional, Callable, List
from twilio.rest import Client

# Metadata defines the available APIs and their trigger words and payload requirements
metadata = {
    "send_sms": {
        "trigger_word": ["send a message", "set a demo", "set up a demo", "send sms", "appointment"],  # Dynamic trigger words
        "sample_payload": {"name": "string"},  # Payload requirement
        "prerequisite": None  # No prerequisite
    },
}

# Event stream to log user actions and intent processor decisions
event_stream = [
    {"event": "user_message", "content": "I want to set up a demo."}
]

def match_trigger_words(user_message: str, trigger_words: List[str]) -> bool:
    """
    Check if any of the trigger words are present in the user's message.
    Returns True if a match is found, otherwise False.
    """
    # Simple matching, allowing case-insensitive substring matching
    for trigger in trigger_words:
        # Use regular expression for word matching, case-insensitive
        if re.search(rf"\b{trigger.lower()}\b", user_message.lower()):
            return True
    return False


def intent_processor(event_stream: List[Dict[str, Any]], metadata: Dict[str, Any]) -> None:
    """
    Process the event stream, detect any intent using metadata trigger words,
    and append an API call to the event stream if a trigger word is detected.
    """
    # Find the latest user message from the event stream
    last_event = next((event for event in reversed(event_stream) if event.get("event") == "user_message"), None)

    if not last_event:
        print("No user message found in event stream.")
        return

    user_message = last_event.get("content", "")

    # Iterate through all functions in metadata and check if trigger words are present
    for api_name, api_metadata in metadata.items():
        trigger_words = api_metadata.get("trigger_word", [])

        # Check if any trigger word matches the user message
        if match_trigger_words(user_message, trigger_words):
            print(f"Intent detected for API call: {api_name}")

            # Append the API call to the event stream
            event_stream.append({
                "intent_processor": "api_call",
                "api_name": api_name,
                "response": {"status": "none"}
            })
            break
    else:
        # No intent detected, use the bot for a general conversation
        bot_response = bot.generate_response(user_message)
        print(f"Bot Response: {bot_response}")

# Function registry to map API names to actual function calls
function_registry: Dict[str, Callable[..., Dict[str, Any]]] = {}

def register_function(api_name: str):
    """
    Decorator to register API functions dynamically into the function registry.
    """
    def decorator(func: Callable[..., Dict[str, Any]]):
        function_registry[api_name] = func
        return func
    return decorator

@register_function("send_sms")
def send_sms(payload: Dict[str, str], account_sid: str, auth_token: str) -> Dict[str, Any]:
    """
    Simulate sending an SMS using the Twilio API.
    """
    print(f"API Call: Sending SMS with payload: {payload}")

    # Initialize Twilio Client with provided credentials
    client = Client(account_sid, auth_token)

    # Simulate sending a message (replace with actual logic for real SMS sending)
    message = client.messages.create(
        body=f"Hello {payload['name']}, this is a test message.",
        from_="+18552060350",  # Replace with a valid Twilio number
        to='+15859538396'  # Replace with the destination number
    )

    print(f"Message SID: {message.sid}")
    response = {"status": f"success: {message.sid}", "model_name": "None"}

    # Log the API call in the event stream
    event_stream.append({"event": "api_call", "api_name": "send_sms", "response": response})
    return response

def find_in_event_stream(key: str) -> Optional[str]:
    """
    Helper function to find a piece of information in the event stream.
    Searches in reverse order for the most recent occurrence of a matching key.
    """
    for event in reversed(event_stream):
        if event.get("event") == "user_message" and key.lower() in event.get("content", "").lower():
            return event.get("content")
    return None

def check_prerequisite(api_name: str) -> bool:
    """
    Check if the prerequisite API call for the given API has been made.
    """
    prerequisite = metadata[api_name]["prerequisite"]
    if not prerequisite:
        return True
    # Check if the prerequisite API call is in the event stream with a success status
    for event in event_stream:
        if event.get("event") == "api_call" and event.get("api_name") == prerequisite and event["response"].get("status") == "success":
            return True
    return False

def resolve_and_execute(account_sid: Optional[str] = None, auth_token: Optional[str] = None) -> Optional[Dict[str, Any]]:
    """
    Resolve dependencies and execute the specified API call dynamically.
    Only runs if the last event in the event_stream has 'intent_processor': 'api_call'.
    """
    # Check if the most recent event is an API call
    api_event = next((event for event in reversed(event_stream) if event.get("intent_processor") == "api_call"), None)

    if not api_event:
        # No api_call in the event stream, do nothing
        print("No API call found in the event stream. Exiting.")
        return None

    # Get the API name from the event
    api_name = api_event.get("api_name")
    if not api_name or api_name not in metadata:
        print(f"Error: API '{api_name}' is not defined in the metadata.")
        return None

    # Check if the prerequisite for the API is met
    if not check_prerequisite(api_name):
        print(f"Error: Prerequisite for '{api_name}' not met.")
        return None

    # Resolve the payload for the API call
    payload = {}
    for key in metadata[api_name]["sample_payload"]:
        value = find_in_event_stream(key)
        if not value:
            # Prompt user for missing information
            value = input(f"Please provide {key}: ")
            event_stream.append({"event": "user_message", "content": f"My {key} is {value}."})
        payload[key] = value.split(" is ")[-1]

    # Dynamically invoke the correct API function from the function registry
    api_function = function_registry.get(api_name)
    if api_function:
        # Call the API function with the resolved payload, and pass account_sid and auth_token if they are required
        if account_sid and auth_token:
            return api_function(payload, account_sid, auth_token)
        else:
            print(f"Error: account_sid and auth_token must be provided for '{api_name}'")
            return None
    else:
        print(f"Error: No function found for API call '{api_name}'.")
        return None


In [81]:
# Example event stream
event_stream = [
    {"event": "user_message", "content": "Hi."},
    {"event": "assistant_message", "content": "Hello! How can I assist you today?"},
    {"event": "user_message", "content": "I want to set up a demo."},
    # {"intent_processor": "api_call", "api_name": "send_sms", "response": {"status": "none"}}
]

# Run the intent processor with the current event_stream and metadata
intent_processor(event_stream, metadata)


Intent detected for API call: send_sms


In [82]:
event_stream

[{'event': 'user_message', 'content': 'Hi.'},
 {'event': 'assistant_message',
  'content': 'Hello! How can I assist you today?'},
 {'event': 'user_message', 'content': 'I want to set up a demo.'},
 {'intent_processor': 'api_call',
  'api_name': 'send_sms',
  'response': {'status': 'none'}}]

In [83]:
# Call resolve_and_execute for the "send_sms" API
result = resolve_and_execute(account_sid, auth_token)

Please provide name: John
API Call: Sending SMS with payload: {'name': 'John'}
Message SID: SM4bdc9222b6bf08b40e047f6423f7636f


In [84]:
event_stream

[{'event': 'user_message', 'content': 'Hi.'},
 {'event': 'assistant_message',
  'content': 'Hello! How can I assist you today?'},
 {'event': 'user_message', 'content': 'I want to set up a demo.'},
 {'intent_processor': 'api_call',
  'api_name': 'send_sms',
  'response': {'status': 'none'}},
 {'event': 'user_message', 'content': 'My name is John.'},
 {'event': 'api_call',
  'api_name': 'send_sms',
  'response': {'status': 'success: SM4bdc9222b6bf08b40e047f6423f7636f',
   'model_name': 'None'}}]